In [1]:
import tensorflow as tf
import numpy as np

## simple linear regression with tensorflow

In [2]:
# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype("float32")
y_data = x_data * 0.1 + 0.3

In [3]:
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b

In [4]:
# optimization, minimize mean sq. error
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

In [5]:
# need to initialize all of the variables
init = tf.initialize_all_variables()

In [6]:
# create a tensor flow session
sess = tf.Session()
# then launch the graph
sess.run(init)

In [7]:
for step in xrange(201):
    sess.run(train)
    if step % 20 == 0:
        print step, sess.run(W), sess.run(b)

0 [ 0.63011295] [ 0.01112298]
20 [ 0.23947884] [ 0.22462909]
40 [ 0.13662867] [ 0.2802068]
60 [ 0.1096191] [ 0.2948021]
80 [ 0.10252607] [ 0.29863498]
100 [ 0.10066337] [ 0.29964155]
120 [ 0.1001742] [ 0.2999059]
140 [ 0.10004573] [ 0.29997531]
160 [ 0.10001201] [ 0.29999352]
180 [ 0.10000315] [ 0.29999831]
200 [ 0.10000082] [ 0.29999956]


## Load the mnist dataset

In [14]:
import os, sys
data_home = os.environ.get('DATA_HOME', None) + "/mnist"
sys.path.insert(0, data_home)

import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [15]:
sess = tf.InteractiveSession()

In [16]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

In [17]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [18]:
sess.run(tf.initialize_all_variables())

In [19]:
# model
y = tf.nn.softmax(tf.matmul(x,W) + b)

# cost function
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

In [20]:
# training function
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [21]:
# train
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [22]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [23]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [24]:
print accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})

0.9092


## multilayer convolutional network

In [25]:
# for initializing model weights

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [26]:

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


In [27]:
# first layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [28]:
# reshape to 4-d tensor, x, image width, image height, color channels
x_image = tf.reshape(x, [-1,28,28,1])

In [29]:
# apply the layer to the input and use the relu activation function
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [30]:
# second layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [31]:
# densly connected layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [32]:
# avoid overfitting using 'dropout', a probability that any given neuron's activation is blocked
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [33]:
# softmax output layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [ ]:
# training

# optimization function
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))

# training step
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# count the num correctly predicted
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

# accuracy function
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# unitialize the session
sess.run(tf.initialize_all_variables())

# batch training
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print "step %d, training accuracy %g"%(i, train_accuracy)
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print "test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})

step 0, training accuracy 0.08
step 100, training accuracy 0.82
step 200, training accuracy 0.9
step 300, training accuracy 1
step 400, training accuracy 0.88
step 500, training accuracy 0.98
step 600, training accuracy 0.94